In [2]:
import numpy as np
import cv2

def initialize_population(image, pop_size=10):
    population = []
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Thresholding to find high-intensity regions
    _, thresh = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    
    # Red color mask
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    red_mask = mask1 + mask2
    
    # Combine masks
    combined_mask = cv2.bitwise_and(thresh, red_mask)
    
    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize population with bounding boxes of detected regions
    for contour in contours[:pop_size]:
        x, y, w, h = cv2.boundingRect(contour)
        population.append((x, y, w, h))
    
    return population


In [3]:
def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Calculate redness
    mask1 = cv2.inRange(hsv_roi, lower_red, upper_red)
    mask2 = cv2.inRange(hsv_roi, lower_red, upper_red)
    red_mask = mask1 + mask2
    red_area = np.sum(red_mask) / (w * h)
    
    # Shape: prefer circular or oval shapes
    aspect_ratio = float(w) / h
    shape_score = 1.0 if 0.75 <= aspect_ratio <= 1.25 else 0.5
    
    # Texture: use LBP to score texture
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    texture_score = np.mean(hist)
    
    # Combine scores
    fitness = red_area * 0.4 + shape_score * 0.3 + texture_score * 0.3
    return fitness


In [4]:
import random

def crossover(parent1, parent2):
    x1, y1, w1, h1 = parent1
    x2, y2, w2, h2 = parent2
    
    child1 = (x1, y2, w1, h2)
    child2 = (x2, y1, w2, h1)
    
    return child1, child2

def mutate(candidate, image_shape, mutation_rate=0.1):
    x, y, w, h = candidate
    if random.random() < mutation_rate:
        x += int(random.gauss(0, 5))
    if random.random() < mutation_rate:
        y += int(random.gauss(0, 5))
    if random.random() < mutation_rate:
        w += int(random.gauss(0, 5))
    if random.random() < mutation_rate:
        h += int(random.gauss(0, 5))
    
    x = np.clip(x, 0, image_shape[1] - 1)
    y = np.clip(y, 0, image_shape[0] - 1)
    w = np.clip(w, 1, image_shape[1] - x)
    h = np.clip(h, 1, image_shape[0] - y)
    
    return (x, y, w, h)

def select(population, fitnesses, num_select):
    selected_indices = np.argsort(fitnesses)[-num_select:]
    selected = [population[i] for i in selected_indices]
    return selected


In [5]:
def genetic_algorithm(image, num_generations=20, pop_size=10, mutation_rate=0.1):
    population = initialize_population(image, pop_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(candidate, image) for candidate in population]
        population = select(population, fitnesses, pop_size // 2)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, image.shape, mutation_rate))
            new_population.append(mutate(child2, image.shape, mutation_rate))
        population = new_population
    best_candidate = max(population, key=lambda candidate: fitness_function(candidate, image))
    return best_candidate


In [6]:
from keras.applications import VGG16
from keras.models import Model

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

def extract_cnn_features(image):
    image = cv2.resize(image, (150, 150))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    features = model.predict(image)
    return features.flatten()

from skimage.feature import local_binary_pattern

def extract_lbp_features(image, candidate):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist


In [8]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"

def load_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label in ['Positive', 'Negative']:
        label_dir = os.path.join(data_dir, label)
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)
    return image_paths, labels

# Load paths and labels for training data
train_image_paths, train_labels = load_image_paths_and_labels(train_path)
val_image_paths, val_labels = load_image_paths_and_labels(val_path)

def initialize_population(image, pop_size=10):
    population = []
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Thresholding to find high-intensity regions
    _, thresh = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    
    # Red color mask
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    red_mask = mask1 + mask2
    
    # Combine masks
    combined_mask = cv2.bitwise_and(thresh, red_mask)
    
    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize population with bounding boxes of detected regions
    for contour in contours[:pop_size]:
        x, y, w, h = cv2.boundingRect(contour)
        population.append((x, y, w, h))
    
    return population

def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Calculate redness
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_roi, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_roi, lower_red, upper_red)
    red_mask = mask1 + mask2
    red_area = np.sum(red_mask) / (w * h)
    
    # Shape: prefer circular or oval shapes
    aspect_ratio = float(w) / h
    shape_score = 1.0 if 0.75 <= aspect_ratio <= 1.25 else 0.5
    
    # Texture: use LBP to score texture
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    texture_score = np.mean(hist)
    
    # Combine scores
    fitness = red_area * 0.4 + shape_score * 0.3 + texture_score * 0.3
    return fitness

def genetic_algorithm(image, num_generations=20, pop_size=10, mutation_rate=0.1):
    population = initialize_population(image, pop_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(candidate, image) for candidate in population]
        population = select(population, fitnesses, pop_size // 2)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, image.shape, mutation_rate))
            new_population.append(mutate(child2, image.shape, mutation_rate))
        population = new_population
    best_candidate = max(population, key=lambda candidate: fitness_function(candidate, image))
    return best_candidate

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

def extract_cnn_features(image):
    image = cv2.resize(image, (150, 150))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    features = model.predict(image)
    return features.flatten()

def extract_lbp_features(image, candidate):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

def combine_features(cnn_features, lbp_features):
    return np.hstack((cnn_features, lbp_features))

def load_images_and_labels(image_paths, labels):
    combined_features = []
    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        detected_rash = genetic_algorithm(image)
        
        cnn_features = extract_cnn_features(image)
        lbp_features = extract_lbp_features(image, detected_rash)
        features = combine_features(cnn_features, lbp_features)
        combined_features.append(features)
    
    return np.array(combined_features), np.array(labels)

# Load and preprocess the dataset
X_train, y_train = load_images_and_labels(train_image_paths, train_labels)
X_val, y_val = load_images_and_labels(val_image_paths, val_labels)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Validate classifier
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)

print(f'Validation Accuracy: {accuracy*100:.2f}%')
print('Confusion Matrix:')
print(cm)


ValueError: Sample larger than population or is negative

In [10]:
import os
import cv2
import numpy as np
import random
from skimage.feature import local_binary_pattern
from keras.applications import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
# Paths to the dataset folders
train_path = "/Users/niteshyadav/Lyme_Disease/train"
test_path = "/Users/niteshyadav/Lyme_Disease/test"
val_path = "/Users/niteshyadav/Lyme_Disease/val"
def load_image_paths_and_labels(data_dir):
    image_paths = []
    labels = []
    for label in ['Positive', 'Negative']:
        label_dir = os.path.join(data_dir, label)
        for image_name in os.listdir(label_dir):
            image_paths.append(os.path.join(label_dir, image_name))
            labels.append(label)
    return image_paths, labels

# Load paths and labels for training data
train_image_paths, train_labels = load_image_paths_and_labels(train_path)
val_image_paths, val_labels = load_image_paths_and_labels(val_path)

def initialize_population(image, pop_size=10):
    population = []
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Thresholding to find high-intensity regions
    _, thresh = cv2.threshold(gray_image, 200, 255, cv2.THRESH_BINARY)
    
    # Red color mask
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_image, lower_red, upper_red)
    
    red_mask = mask1 + mask2
    
    # Combine masks
    combined_mask = cv2.bitwise_and(thresh, red_mask)
    
    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize population with bounding boxes of detected regions
    for contour in contours[:pop_size]:
        x, y, w, h = cv2.boundingRect(contour)
        population.append((x, y, w, h))
    
    # If not enough regions, fill with random regions
    while len(population) < pop_size:
        h, w = gray_image.shape
        x = random.randint(0, w-1)
        y = random.randint(0, h-1)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        population.append((x, y, width, height))
    
    return population

def fitness_function(candidate, image):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
    # Calculate redness
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])
    mask1 = cv2.inRange(hsv_roi, lower_red, upper_red)
    
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv_roi, lower_red, upper_red)
    red_mask = mask1 + mask2
    red_area = np.sum(red_mask) / (w * h)
    
    # Shape: prefer circular or oval shapes
    aspect_ratio = float(w) / h
    shape_score = 1.0 if 0.75 <= aspect_ratio <= 1.25 else 0.5
    
    # Texture: use LBP to score texture
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    texture_score = np.mean(hist)
    
    # Combine scores
    fitness = red_area * 0.4 + shape_score * 0.3 + texture_score * 0.3
    return fitness

def crossover(parent1, parent2):
    x1, y1, w1, h1 = parent1
    x2, y2, w2, h2 = parent2
    child1 = (x1, y2, w1, h2)
    child2 = (x2, y1, w2, h1)
    return child1, child2

def mutate(candidate, image_shape, mutation_rate=0.1):
    if random.random() < mutation_rate:
        h, w, _ = image_shape
        x, y, w, h = candidate
        x = random.randint(0, w-1)
        y = random.randint(0, h-1)
        width = random.randint(10, 50)
        height = random.randint(10, 50)
        candidate = (x, y, width, height)
    return candidate

def select(population, fitnesses, num_to_select):
    selected = list(zip(population, fitnesses))
    selected.sort(key=lambda x: x[1], reverse=True)
    return [candidate for candidate, fitness in selected[:num_to_select]]

def genetic_algorithm(image, num_generations=20, pop_size=10, mutation_rate=0.1):
    population = initialize_population(image, pop_size)
    for generation in range(num_generations):
        fitnesses = [fitness_function(candidate, image) for candidate in population]
        population = select(population, fitnesses, pop_size // 2)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, image.shape, mutation_rate))
            new_population.append(mutate(child2, image.shape, mutation_rate))
        population = new_population
    best_candidate = max(population, key=lambda candidate: fitness_function(candidate, image))
    return best_candidate

# Load pre-trained VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

def extract_cnn_features(image):
    image = cv2.resize(image, (150, 150))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    features = model.predict(image)
    return features.flatten()

def extract_lbp_features(image, candidate):
    x, y, w, h = candidate
    roi = image[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray_roi, P=8, R=1, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 27), range=(0, 26))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)
    return hist

def combine_features(cnn_features, lbp_features):
    return np.hstack((cnn_features, lbp_features))

def load_images_and_labels(image_paths, labels):
    combined_features = []
    for image_path, label in zip(image_paths, labels):
        image = cv2.imread(image_path)
        detected_rash = genetic_algorithm(image)
        
        cnn_features = extract_cnn_features(image)
        lbp_features = extract_lbp_features(image, detected_rash)
        features = combine_features(cnn_features, lbp_features)
        combined_features.append(features)
    
    return np.array(combined_features), np.array(labels)

# Load and preprocess the dataset
X_train, y_train = load_images_and_labels(train_image_paths, train_labels)
X_val, y_val = load_images_and_labels(val_image_paths, val_labels)

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Validate classifier
y_pred = clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
cm = confusion_matrix(y_val, y_pred)

print(f'Validation Accuracy: {accuracy*100:.2f}%')
print('Confusion Matrix:')
print(cm)




1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 47ms/step


...

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 48ms/step
Validation Accuracy: 84.39%
Confusion Matrix:
[[407   5]
 [ 74  20]]


In [11]:
test_image_paths, test_labels = load_image_paths_and_labels(test_path)
# Load and preprocess the dataset

X_test, y_test = load_images_and_labels(test_image_paths, test_labels)



y_test = le.transform(y_test)


# Validate classifier
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print(f'Validation Accuracy: {accuracy*100:.2f}%')
print('Confusion Matrix:')
print(cm)

1/1 [==============================] - 0s 47ms/step


1/1 [==============================] - 0s 48ms/step
Validation Accuracy: 82.05%
Confusion Matrix:
[[399  13]
 [ 78  17]]


In [12]:
# Predict labels for training data
y_train_pred = clf.predict(X_train)

# Compute and print training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Training Accuracy: {train_accuracy*100:.2f}%')

# If you also want to see the confusion matrix for training data
train_cm = confusion_matrix(y_train, y_train_pred)
print('Training Confusion Matrix:')
print(train_cm)


Training Accuracy: 99.75%
Training Confusion Matrix:
[[3291    3]
 [   7  745]]


In [16]:
!pip install keras.wrappers

ERROR: Could not find a version that satisfies the requirement keras.wrappers (from versions: none)
ERROR: No matching distribution found for keras.wrappers

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


# reducing Negative images
